In [1]:
from facebook import GraphAPI
from facebook_database_helper import SimpleFacebookDBHelper
from facebook_database_helper import NameRelation

from elasticsearch import Elasticsearch

import time as t
import requests
import re

In [2]:
def time():
    return int(round(t.time() * 1000))

In [3]:
def constant(func):
    # noinspection SpellCheckingInspection
    def funcset(self, value):
        raise TypeError

    # noinspection SpellCheckingInspection
    def funcget(self):
        return func()

    return property(funcget, funcset)


# noinspection PyPep8Naming,PyMethodParameters
class _Const(object):
    @constant
    def FACEBOOK_TOKEN():
        return "1769775703259571|736fc7f9c5dc31707d40709a1d37813b"

    @constant
    def FACEBOOK_POSTS_COUNT():
        return 3000

    @constant
    def ES_INDEX():
        return "fb_group_posts"

    @constant
    def ES_POSTS_DOC_TYPE():
        return "post"
    
    @constant
    def ES_NAMES_RELATIONS_DOC_TYPE():
        return "name_relations"
    
    @constant
    def ES_BULK_SIZE():
        return 500


CONST = _Const()

In [4]:
class Group(object):
    def __init__(self, name, id, domain):
        self.name = name
        self.id = id
        self.domain = domain
        
groups = [
#     Group(name="CNN Politics", id="219367258105115", domain="politics"),
#     Group(name="SinoRuss", id="1565161760380398", domain="politics"),
#     Group(name="Politics & Sociology", id="1616754815303974", domain="politics"),
#     Group(name="CNN Money", id="6651543066", domain="finances"),
#     Group(name="MTV", id="7245371700", domain="music"),
#     Group(name="CNET", id="7155422274", domain="tech"),
#     Group(name="TechCrunch", id="8062627951", domain="tech"),
#     Group(name="Sport Addicts", id="817513368382866", domain="sport"),
#     Group(name="Pokemon GO", id="1745029562403910", domain="pokemon go")
]

In [5]:
database = Elasticsearch()

In [6]:
graph = GraphAPI(access_token=CONST.FACEBOOK_TOKEN)

In [7]:
fdb = SimpleFacebookDBHelper(
    es=database, 
    index=CONST.ES_INDEX, 
    post_doc_type=CONST.ES_POSTS_DOC_TYPE, 
    name_relation_doc_type=CONST.ES_NAMES_RELATIONS_DOC_TYPE)

In [8]:
def pull_posts_from_pack(posts):
    res = []

    for p in posts['data']:
        res.append(p)

    return res

In [9]:
def prepare_get_posts_request(posts):
    return posts['paging']['next'].replace("limit=25", "limit=100")

In [10]:
def load_next_posts(posts, max_count):
    result = []
    count = 0

    while True:
        if count > max_count:
            break

        try:
            result.extend(pull_posts_from_pack(posts))

            request = prepare_get_posts_request(posts)

            s_time = time()
            posts = requests.get(request).json()
            f_time = time()
            
            posts_count = len(posts['data'])
            count += posts_count
            
            print "Время загрузки пака постов ->", (f_time - s_time), "|", "кол-во:", posts_count, "|", "всего:", count
        except KeyError:
            break

    print "Общее кол-во загруженных постов группы ->", count

    return result

In [11]:
for group in groups:
    s_time = time()
    group_json = graph.get_object(group.id)
    f_time = time()
    
    print 'Время загрузки данных [', group_json['name'], '] группы ->', (f_time - s_time)
    
    s_time = time()
    first_posts_pack = graph.get_connections(group_json['id'], 'feed')
    f_time = time()
    
    print 'Время загрузки первого пака постов ->', (f_time - s_time)
    
    posts = load_next_posts(posts=first_posts_pack, max_count=CONST.FACEBOOK_POSTS_COUNT)
    
    s_time = time()
    fdb.save_posts(group.name, group.domain, posts)
    f_time = time()
    
    print "Время сохранение постов в БД ->", (f_time - s_time)
    print "-------------------------"

In [32]:
FIRST_LAST_NAME_PATTERN = "[A-Z]{1}[a-z]+\s+[A-Z]{1}[a-z]+"

In [33]:
fdb.delete_all_name_relations()

21129

In [34]:
posts = fdb.get_all_posts()
name_relations = []

for post in posts:
    message = post['_source']['message']
    names = re.findall(FIRST_LAST_NAME_PATTERN, message)

    if names:
        id = post['_id']

        for name in names:
            name = name.replace(".", "")
            relation = NameRelation(fl_name=name, post_id=id)
            name_relations.append(relation)

for i in range(min(len(name_relations), 25)):
    relation = name_relations[i]
    print relation.fl_name
    
len(name_relations)

South China
United States
If China
The Beijing
Foreign Policy
Pacific Ocean
Manifest Destiny
Pacific Power
Johnny Come
Johnny Come
Hilary Clinton
Asia Pivot
American Empire
Zionist Jewish
Wall Street
Big Oil
Big Business
Lib American
South China
United States
If China
The Beijing
Foreign Policy
Pacific Ocean
Manifest Destiny


29003

In [35]:
s_time = time()
deleted_posts = fdb.delete_all_name_relations()
f_time = time()

print "Время удаления старых отношений 'сообщение <=> имя' ->", (f_time - s_time)
print "Удалено старых постов:", deleted_posts

Время удаления старых отношений 'сообщение <=> имя' -> 2
Удалено старых постов: 0


In [36]:
s_time = time()
fdb.save_name_relations(name_relations)
f_time = time()

print "Время сохранения новых отношений 'сообщение <=> имя' ->", (f_time - s_time)

Время сохранения новых отношений 'сообщение <=> имя' -> 12462


In [37]:
searched_names = [
    "John Kerry"
]

for name in searched_names:
    messages = fdb.find_messages_by_fl_name(name)
    print "Имя [", name, "] найдено соответстивий:", len(messages)
    
    if messages:
        print "Пример сообщения:", messages[0]

Имя [ John Kerry ] найдено соответстивий: 25
Пример сообщения: This piece of garbage spin from the New York Times reporters Jonathan Soble and Choe Sang-Hun equating this deal brokered by John Kerry, U.S. Secretary of State as "Nixonian," is absurd. In the case of Nixon's rapprochment with Communist China, no women were violated, abused and subjected to mass rapes by beasts from Japan's invading Imperial Army. How asinine could this comparison be.... as if Asian women are made to be "sex slaves -- Geisha girls, and sing-song girls for the schmucks!" Fucking racists from the NYT.
This was a betrayal by South Korean woman president Park Geun Hye, and very obviously her South Korean government is deeply infiltrated by obsequious U.S. agents, including Defense and the Foreign Ministry.
South Korea is a vassal-minion state of Pax Americana..... a paltry U.S.$ 8.3 million to buy closure to this horror of sex enslavement of Korean women during World War II under the pricks of Japanese war cri